<a href="https://colab.research.google.com/github/Vishal0Nagarajan/Flight-Delay-Prediction/blob/master/DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
import pandas as pd
import os
import json
import dask.dataframe as dd
import time
from datetime import datetime

In [ ]:
l = list()
req_airports = ['ATL','CLT','DEN','DFW','EWR','IAH','JFK','LAS','LAX','MCO','MIA','ORD','PHX','SEA','SFO']
path1 = 'drive/My Drive/Flight Data/2016_original/2016/'
for root, dirs, files in os.walk(path1):
  for file_ in files:
    if file_.endswith(".csv"):
      print(path1+file_)
      df = dd.read_csv(os.path.join(root, file_), sep = ',', error_bad_lines = False, dtype = 'unicode')
      X = df[df['Origin'].isin(req_airports) & df['Dest'].isin(req_airports)] 
      l.append(X[['Origin','Dest','FlightDate','Quarter','Year','Month','DayofMonth','DepTime','DepDel15','CRSDepTime','DepDelayMinutes','OriginAirportID','DestAirportID','ArrTime','CRSArrTime','ArrDel15','ArrDelayMinutes']])
path2 = 'drive/My Drive/Flight Data/2017_original/2017/'
for root, dirs, files in os.walk(path2):
  for file_ in files:
    if file_.endswith(".csv"):
      df = dd.read_csv(os.path.join(root, file_), sep = ',', error_bad_lines = False, dtype = 'unicode')
      X = df[df['Origin'].isin(req_airports) & df['Dest'].isin(req_airports)]
      l.append(X[['Origin','Dest','FlightDate','Quarter','Year','Month','DayofMonth','DepTime','DepDel15','CRSDepTime','DepDelayMinutes','OriginAirportID','DestAirportID','ArrTime','CRSArrTime','ArrDel15','ArrDelayMinutes']])
data = pd.concat(l)
data = data.dropna()
data = data.drop_duplicates(keep = 'first')
data = data.reset_index(drop = True)
l.clear()
data

ValueError: ignored

In [ ]:
path3 = 'drive/My Drive/weather/'
li = list()
for a in range(len(os.listdir(path3))):
  f = path3+os.listdir(path3)[a]+'/'
  for root, dirs, files in os.walk(f):
    for file_ in files:
      if (file_.startswith('2016') | file_.startswith('2017')):
        print(f+file_)
        dat = json.load(open(f+file_, 'rb'))
        del dat['data']['request']
        req_weather = ['windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM', 'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph', 'tempF', 'WindChillF', 'humidity', 'date', 'time', 'airport']
        hourly_keys=list(dat['data']['weather'][0]['hourly'][0].keys())
        hourly_req = [i for i in hourly_keys if i in req_weather]
        myDict = {key:None for key in hourly_req}
        myDict['date'] = None
        myDict['airport'] = str(path3+os.listdir(path3)[a][-3:])     
        for k in range(31):
          try:
            myDict['date'] = dat['data']['weather'][k]['date']
          except IndexError:
            break
          for i in range(24):
            myDict.update({key : dat['data']['weather'][0]['hourly'][i][key] for key in hourly_req})
            d2 = pd.DataFrame(myDict, index = [0])
            d2 = d2.reset_index(drop = True)
            li.append(d2)
d_final = pd.concat(li)
d_final = d_final.reset_index(drop = True)
d_final['airport'] = d_final['airport'].apply(lambda x : x[-3:])
d_final

FileNotFoundError: ignored

In [ ]:
data.to_csv('drive/My Drive/data.csv')
d_final.to_csv('drive/My Drive/d_final.csv')

In [ ]:
data = pd.read_csv('drive/My Drive/data.csv', index_col = 0)
d_final = pd.read_csv('drive/My Drive/d_final.csv', index_col = 0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
data['DepTime'] = data['DepTime'].apply(lambda x : str(int(int(x)/100)) if str(int(int(x)/100))!='24' else '0')
data['ArrTime'] = data['ArrTime'].apply(lambda x : str(int(int(x)/100)) if str(int(int(x)/100))!='24' else '0')
d_final['time'] = d_final['time'].apply(lambda x : str(int(int(x)/100)) if str(int(int(x)/100))!='24' else '0')
df_merge = pd.merge(data, d_final, left_on = ['FlightDate', 'Origin', 'DepTime'], right_on = ['date', 'airport', 'time'], how = 'left')
df_merge_final = pd.merge(df_merge, d_final, left_on = ['FlightDate', 'Dest', 'ArrTime'], right_on = ['date', 'airport', 'time'], how = 'left')
df_merge_final

,Origin,Dest,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,windspeedKmph_x,DewPointF_x,cloudcover_x,precipMM_x,pressure_x,WindGustKmph_x,visibility_x,weatherCode_x,tempF_x,WindChillF_x,winddirDegree_x,humidity_x,time_x,date_x,airport_x,windspeedKmph_y,DewPointF_y,cloudcover_y,precipMM_y,pressure_y,WindGustKmph_y,visibility_y,weatherCode_y,tempF_y,WindChillF_y,winddirDegree_y,humidity_y,time_y,date_y,airport_y
0,LAX,LAS,2016-11-05,4,2016,11,5,8,0.0,900,0.0,12892,12889,10,1019,0.0,0.0,5,56,18,0.0,1016,6,10,113,64,63,112,79,8,2016-11-05,LAX,3,38,65,0.0,1013,4,10,116,58,56,65,54,10,2016-11-05,LAS
1,DFW,ATL,2016-11-05,4,2016,11,5,13,0.0,1345,0.0,11298,10397,16,1656,0.0,0.0,21,67,57,0.0,1015,25,10,116,77,76,180,76,13,2016-11-05,DFW,13,59,0,0.0,1022,14,10,113,82,81,110,48,16,2016-11-05,ATL
2,ATL,CLT,2016-11-05,4,2016,11,5,13,0.0,1340,0.0,10397,11057,14,1457,0.0,0.0,12,60,0,0.0,1023,14,10,113,78,76,109,59,13,2016-11-05,ATL,5,59,48,0.0,1025,6,10,116,72,70,176,67,14,2016-11-05,CLT
3,CLT,ATL,2016-11-05,4,2016,11,5,15,0.0,1540,0.0,11057,10397,16,1657,0.0,0.0,5,59,34,0.0,1025,6,10,116,74,72,192,64,15,2016-11-05,CLT,13,59,0,0.0,1022,14,10,113,82,81,110,48,16,2016-11-05,ATL
4,LAX,SEA,2016-11-05,4,2016,11,5,7,0.0,755,0.0,12892,14747,10,1050,0.0,1.0,5,57,20,0.0,1016,6,10,116,64,63,97,81,7,2016-11-05,LAX,15,49,100,0.2,1012,24,6,176,52,48,202,89,10,2016-11-05,SEA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851428,IAH,DFW,2017-10-28,4,2017,10,28,10,0.0,1030,0.0,12266,11298,11,1151,0.0,0.0,6,62,21,0.0,1015,8,10,113,79,79,66,56,10,2017-10-28,IAH,11,53,3,0.0,1016,13,10,113,76,76,141,46,11,2017-10-28,DFW
1851429,DFW,IAH,2017-10-07,4,2017,10,7,13,0.0,1340,0.0,11298,12266,14,1456,0.0,0.0,12,54,3,0.0,1015,13,10,113,81,81,153,40,13,2017-10-07,DFW,7,65,36,0.0,1013,8,10,116,86,86,99,49,14,2017-10-07,IAH
1851430,DFW,IAH,2017-10-14,4,2017,10,14,13,0.0,1340,0.0,11298,12266,14,1456,0.0,0.0,12,54,3,0.0,1015,13,10,113,81,81,153,40,13,2017-10-14,DFW,7,65,36,0.0,1013,8,10,116,86,86,99,49,14,2017-10-14,IAH
1851431,DFW,IAH,2017-10-21,4,2017,10,21,13,0.0,1340,0.0,11298,12266,14,1456,0.0,0.0,12,54,3,0.0,1015,13,10,113,81,81,153,40,13,2017-10-21,DFW,7,65,36,0.0,1013,8,10,116,86,86,99,49,14,2017-10-21,IAH


In [ ]:
df_merge_final.to_csv('drive/My Drive/DataPreprocessing.csv')